<a href="https://colab.research.google.com/github/Madalina0303/Licenta-Depresie/blob/main/Alti_Clasificatori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from xml.dom.minidom import parse
import os
import re
import string
import numpy as np
import pandas as pd
import math
from nltk.corpus import stopwords

In [3]:
contractions_dict = { "ain't": "are not","'s":" is","aren't": "are not",
                     "can't": "cannot","can't've": "cannot have",
                     "'cause": "because","could've": "could have","couldn't": "could not",
                     "couldn't've": "could not have", "didn't": "did not","doesn't": "does not",
                     "don't": "do not","hadn't": "had not","hadn't've": "had not have",
                     "hasn't": "has not","haven't": "have not","he'd": "he would",
                     "he'd've": "he would have","he'll": "he will", "he'll've": "he will have",
                     "how'd": "how did","how'd'y": "how do you","how'll": "how will",
                     "i'd": "i would", "i'd've": "i would have","i'll": "i will",
                     "i'll've": "i will have","i'm": "i am","I've": "i have", "isn't": "is not",
                     "it'd": "it would","it'd've": "it would have","it'll": "it will",
                     "it'll've": "it will have", "let's": "let us","ma'am": "madam",
                     "mayn't": "may not","might've": "might have","mightn't": "might not", 
                     "mightn't've": "might not have","must've": "must have","mustn't": "must not",
                     "mustn't've": "must not have", "needn't": "need not",
                     "needn't've": "need not have","o'clock": "of the clock","oughtn't": "ought not",
                     "oughtn't've": "ought not have","shan't": "shall not","sha'n't": "shall not",
                     "shan't've": "shall not have","she'd": "she would","she'd've": "she would have",
                     "she'll": "she will", "she'll've": "she will have","should've": "should have",
                     "shouldn't": "should not", "shouldn't've": "should not have","so've": "so have",
                     "that'd": "that would","that'd've": "that would have", "there'd": "there would",
                     "there'd've": "there would have", "they'd": "they would",
                     "they'd've": "they would have","they'll": "they will",
                     "they'll've": "they will have", "they're": "they are","they've": "they have",
                     "to've": "to have","wasn't": "was not","we'd": "we would",
                     "we'd've": "we would have","we'll": "we will","we'll've": "we will have",
                     "we're": "we are","we've": "we have", "weren't": "were not","what'll": "what will",
                     "what'll've": "what will have","what're": "what are", "what've": "what have",
                     "when've": "when have","where'd": "where did", "where've": "where have",
                     "who'll": "who will","who'll've": "who will have","who've": "who have",
                     "why've": "why have","will've": "will have","won't": "will not",
                     "won't've": "will not have", "would've": "would have","wouldn't": "would not",
                     "wouldn't've": "would not have","y'all": "you all", "y'all'd": "you all would",
                     "y'all'd've": "you all would have","y'all're": "you all are",
                     "y'all've": "you all have", "you'd": "you would","you'd've": "you would have",
                     "you'll": "you will","you'll've": "you will have", "you're": "you are",
                     "you've": "you have"}

In [4]:
contractions_re=re.compile('(%s)' % '|'.join(contractions_dict.keys()))
def expand_contractions(text,contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, text)

In [12]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
stop_words.add('subject')
stop_words.add('http')
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])


In [7]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [8]:
def parse_file(filename):
    doc = parse(filename, )
    titles = doc.getElementsByTagName("TITLE")
    posts = doc.getElementsByTagName("TEXT")
    messages = []
    for index, title in enumerate(titles):
        message = title.firstChild.nodeValue + posts[index].firstChild.nodeValue
        message = expand_contractions(message).lower()
        message= re.sub('[%s]' % re.escape(string.punctuation), '' , message)
        message = re.sub('W*dw*','',message)
        message = remove_stopwords(message)
        message = re.sub('(http[s]?S+)|(w+.[A-Za-z]{2,4}S*)', '', message)
        message = lemmatize_words(message)
        message  = re.sub(' +', ' ', message)
        messages.append(message.strip())
        
    return messages


In [9]:
 from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# dam append sa facem un vector mare din toate postarile, oare cum ar fi cu o matrice ??

In [10]:
def get_data_post_list(dir_name):
    users_posts_list = []
    for root, dir, files in os.walk(dir_name):
        for filename in files:
            if ".xml" in filename:
                nr = re.findall("[0-9]+", filename)
                users_posts_list.append(parse_file(root+"/"+filename))

    return users_posts_list

In [13]:
all_data = get_data_post_list("drive/MyDrive/data")
x_train = all_data[:60]
x_test = all_data[60:]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1000)
 # Stop words are common words in English that don't tell us anything about the polarity of a review.
    # Such words include the, that, and a
# Converts a collection of text documents to a matrix of token counts
# max_features = maximum number of words we'd like to have in our bag of words model
X = []
for i in all_data:
  X.append(cv.fit_transform(i).toarray())

In [ ]:
cv.get_feature_names()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer()
X = tf_transformer.fit_transform(X).toarray()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(max_features =1000)
X = []
for i in all_data:
  X.append(tfidfVectorizer.fit_transform(i).toarray())
  


In [ ]:
vectorizer = TfidfVectorizer(max_features = 5000) 
for i in all_data:
  vectorizer.fit(i) 
  X.append(tfidfVectorizer.transform(i).toarray())

# Xtrain = vectorizer.transform(all_data)
print(len(X[0]))
print(len(X[1]))

In [15]:
# print(len(X[1]))
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 76 kB/s 


In [ ]:
print(X[1])

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB

# Naive Bayes is a statistical classification technique based on Bayes Theorem
# common classifier used in sentiment analysis is the Naive Bayes Classifier.
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier # this is experimental
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
classifiers = [GradientBoostingClassifier(),GaussianNB(),HistGradientBoostingClassifier(),
               RandomForestClassifier(),LogisticRegression(),XGBClassifier(),LGBMClassifier(),
               CatBoostClassifier(verbose=0)]

In [ ]:
import json
with open('drive/MyDrive/Licenta/reponses.json') as json_file:
    responses = json.load(json_file)
  
y_train = responses["Sadness"]
import numpy as np
# one hot target 
# momentan avem 4 raspunsuri de la 0 la 3 facem cu I4 la celelalte mai speciale facem cu I8
print(y_train)
# trg = [[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,0.0,1.0]]
# y_train_target = []
# for j in y_train:
#   y_train_target.append(trg[j])
# y_1 = np.array(y_train_target)
# print(type(y_train_target[0][0]))

y_test = y_train[60:]
y_train = y_train[0:60]

[1, 1, 1, 1, 2, 0, 1, 0, 1, 2, 1, 0, 1, 1, 1, 0, 0, 2, 1, 2, 1, 1, 0, 1, 0, 3, 1, 1, 1, 0, 3, 2, 1, 1, 3, 1, 1, 1, 0, 1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 0, 1, 2, 1, 2, 2, 1, 1, 3, 1, 0, 1, 1, 2, 2, 0, 1, 0, 0, 3, 1, 1, 0, 1, 1, 1, 3, 2, 3, 1]


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
for classifier in classifiers:
    classifier.fit(x_train,y_train)
    print(f'The {classifier}  Accuracy  is {accuracy_score(y_test,classifier.predict(X)) }' )



```
# Acesta are format de cod
```



In [17]:

from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 100000
oov_token = "<OOV>"
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(x_train)

In [18]:
x_train_sequences = tokenizer.texts_to_sequences(x_train)
x_test_sequences = tokenizer.texts_to_sequences(x_test)


In [19]:
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [20]:
# max_length = 150530
padding_type = "post"
trunction_type="post"
x_train_padded = pad_sequences(x_train_sequences, padding=padding_type,
                       truncating=trunction_type,maxlen = 1218)
x_test_padded = pad_sequences(x_test_sequences,
                               padding=padding_type, truncating=trunction_type,maxlen = 1218)

In [21]:
def get_train_test_data(category):
  with open('drive/MyDrive/Licenta/reponses.json') as json_file:
    responses = json.load(json_file)
  y_train = responses[category]
  
  return y_train[0:60], y_train[60:]

In [22]:
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score
def clasificatori(y_train,y_test):
  all_r = []
  for classifier in classifiers:
      classifier.fit(x_train_padded,y_train)
      all_r.append(classifier.predict(x_test_padded )) 
  return all_r
      # print(f'The {classifier}  Accuracy  is {accuracy_score(y_test,classifier.predict(x_test_padded )) }' )

In [23]:
import re
def get_categories(file):
  with open(file) as f:
    data = f.read()
    questions = re.findall("[0-9]+\.\s*[a-zA-Z\s-]+\s*\n", data)
  
  for i, elem in enumerate(questions):
    questions[i] = elem.split("\n")[0]
    questions[i] = re.split("[0-9]+[\.\s]+",questions[i])[1]

  print(questions)
  return questions

In [24]:
all_categories = get_categories("drive/MyDrive/chestionar.txt")
rez_0 = dict()
rez_1 = dict()
rez_2 = dict()
rez_3 = dict()
rez_4 = dict()
rez_5 = dict()
rez_6 = dict()
rez_7 = dict()

['Sadness', 'Pessimism', 'Past Failure', 'Loss of Pleasure', 'Guilty Feelings', 'Punishment Feelings', 'Self-Dislike', 'Self-Criticalness', 'Suicidal Thoughts or Wishes', 'Crying', 'Agitation', 'Loss of Interest', 'Indecisiveness', 'Worthlessness', 'Loss of Energy', 'Changes in Sleeping Pattern', 'Irritability', 'Changes in Appetite', 'Concentration Difficulty', 'Tiredness or Fatigue', 'Loss of Interest in Sex']


In [25]:
import json
def dump_to_json(file_name,data_dict_param):
  with open(file_name, "w") as json_file:
    json.dump(data_dict_param, json_file)


In [26]:
for c in all_categories:
  y_train,y_test = get_train_test_data(c)
  raspuns = clasificatori(y_train,y_test)
  rez_0[c] = raspuns[0]
  rez_1[c] = raspuns[1]
  rez_2[c] = raspuns[2]
  rez_3[c] = raspuns[3]
  rez_4[c] = raspuns[4]
  rez_5[c] = raspuns[5]
  rez_6[c] = raspuns[6]
  rez_7[c] = raspuns[7]

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [40]:
def extract_ans(rezultate,path):
  all_responses = dict()
  for i in range(61,81):
    all_responses[str(i)]= []
  for c in all_categories:
    r = rezultate[c]
    print(r)
    for index,o in enumerate(r):
      all_responses[str(61+index)].append(o.item())
    dump_to_json("drive/MyDrive/"+path+".json",all_responses)


In [51]:
all_alg_responses = []
def load_ok(path):
  with open('drive/MyDrive/'+path+".json") as json_file:
      data = json.load(json_file)
  all_alg_responses.append(data)

In [50]:
print(data)

{'61': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '62': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '63': [1, 1, 0, 1, 0, 1, 3, 2, 2, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '64': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '65': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '66': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '67': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '68': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '69': [2, 3, 0, 1, 0, 1, 3, 0, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 2, 1], '70': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '71': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '72': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '73': [2, 1, 0, 2, 0, 2, 3, 2, 3, 2, 1, 2, 0, 1, 2, 4, 1, 5, 0, 1, 1], '74': [1, 1, 0, 1, 0, 1, 3, 2, 3, 2, 1, 2, 1, 1, 2, 4, 1, 5, 0, 1, 1], '75':

In [39]:
for index,o in enumerate(rez_0['Sadness']):
  print(type(o.item()))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>


In [52]:
load_ok("Alg0")
load_ok("Alg1")
load_ok("Alg2")
load_ok("Alg3")
load_ok("Alg4")
load_ok("Alg5")
load_ok("Alg6")
load_ok("Alg7")

In [ ]:
extract_ans(rez_0,"Alg0")
extract_ans(rez_1,"Alg1")
extract_ans(rez_2,"Alg2")
extract_ans(rez_3,"Alg3")
extract_ans(rez_4,"Alg4")
extract_ans(rez_5,"Alg5")
extract_ans(rez_6,"Alg6")
extract_ans(rez_7,"Alg7")

In [42]:
def get_correct_responses():
  correct = dict()
  for i in range(61,81):
    correct[str(i)]= []
  response  = "drive/MyDrive/Licenta/response.txt"
  transform_resp = {"1a":1, "1b":2, "2a":3, "2b":4, "3a":5, "3b":6}
  with open(response) as rsp:
    data = rsp.readlines()[60:]
    for index,line in enumerate(data):
      res = line[:-1].split(" ")[1:]
      for j in res:
        if len(j)<=1:
          correct[str(61+index)].append(int(j))
        else:
            correct[str(61+index)].append(int(transform_resp[j]))
    return correct

corr = get_correct_responses()
print(corr)

{'61': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0], '62': [1, 0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 1, 2, 1, 0, 2, 0, 1, 1, 1, 0], '63': [1, 2, 3, 1, 1, 0, 3, 2, 1, 3, 2, 2, 3, 2, 2, 3, 2, 2, 1, 1, 1], '64': [2, 3, 2, 3, 1, 2, 3, 2, 1, 0, 3, 3, 3, 3, 3, 2, 1, 3, 2, 3, 2], '65': [2, 2, 3, 2, 3, 2, 2, 3, 1, 3, 2, 2, 3, 3, 3, 3, 2, 0, 1, 2, 0], '66': [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 2, 2, 1, 2], '67': [1, 1, 3, 2, 3, 2, 3, 3, 2, 1, 1, 1, 3, 3, 1, 0, 1, 6, 2, 3, 1], '68': [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0], '69': [0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 4, 1, 4, 2, 2, 1], '70': [3, 2, 2, 1, 2, 2, 2, 3, 1, 1, 2, 2, 2, 2, 2, 4, 2, 1, 2, 1, 0], '71': [1, 1, 0, 3, 1, 3, 2, 2, 0, 2, 2, 3, 1, 1, 3, 4, 0, 6, 2, 3, 3], '72': [1, 1, 0, 1, 0, 2, 1, 1, 0, 2, 1, 1, 3, 2, 0, 6, 0, 1, 2, 1, 0], '73': [0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 0, 0], '74': [1, 2, 1, 1, 0, 0, 0, 0, 1, 3, 1, 2, 1, 0, 2, 5, 2, 2, 2, 2, 2], '75':

In [43]:
# all_alg_responses = [rez_0,rez_1,rez_2,rez_3,rez_4,rez_5,rez_6,rez_7]

In [44]:
def hr(l1, l2):
  c = 0
  for index, e in enumerate(l1):
    if e == l2[index]:
      c += 1
  return c/21

In [45]:
def ahr(all_alg_responses):
  ahr_list = []
  for all_responses  in all_alg_responses:
    s = 0
    for i in range(61,81):
      l1 = all_responses[str(i)]
      l2 = corr[str(i)]
      s += hr(l1,l2)
    ahr_list.append(s/20)
  return ahr_list

In [54]:
ahr(all_alg_responses)
# print(all_alg_responses)

[0.3095238095238095,
 0.1928571428571429,
 0.37142857142857144,
 0.3261904761904762,
 0.22142857142857145,
 0.35714285714285715,
 0.3666666666666666,
 0.34523809523809523]

In [55]:
def dict_transform(elem):
  tr_dict={"0":0,"1":1,"2":1,"3":2,"4":2,"5":3,"6":3}
  return tr_dict[str(elem)]

In [56]:
def cr(l1,l2):
  s = 0
  # if mad == 7:
  #   new_l1 = transform(l1)
  #   new_l2 = transform(l2)
  for index, e in enumerate(l1):
    if index == 15 or index == 17:
      mad = 7
      e = dict_transform(e)
      l2[index] = dict_transform(l2[index])
    else:
      mad = 4
    ad = abs(e-l2[index]) 
    cr = (mad-ad)/mad
    s = s+cr
  return s/21

In [57]:
def acr(all_alg_responses):
  acr_list = []
  for all_responses  in all_alg_responses:
    s = 0
    for i in range(61,81):
      l1 = all_responses[str(i)]
      l2 = corr[str(i)]
      s += cr(l1,l2)
    acr_list.append(s/20)
  return acr_list

In [58]:
acr(all_alg_responses)

[0.7641156462585033,
 0.6876700680272109,
 0.8052721088435375,
 0.7942176870748299,
 0.6976190476190476,
 0.8073979591836734,
 0.8034863945578232,
 0.8056122448979594]

In [59]:
def dodl(l1,l2):
  s1 = 0
  s2 = 0
  for index, e in enumerate(l1):
    if index == 15 or index == 17:
      e = dict_transform(e)
      l2[index] = dict_transform(l2[index])
    s1 += e
    s2 += l2[index]
  ad_overall = abs(s1-s2)
  return (63-ad_overall)/63

In [60]:
def adodl(all_alg_responses):
  adodl_list = []
  for all_responses  in all_alg_responses:
    s = 0
    for i in range(61,81):
      l1 = all_responses[str(i)]
      l2 = corr[str(i)]
      s += dodl(l1,l2)
    adodl_list.append(s/20)
  return adodl_list

In [61]:
adodl(all_alg_responses)

[0.8412698412698413,
 0.815873015873016,
 0.8134920634920636,
 0.8380952380952381,
 0.746825396825397,
 0.8468253968253968,
 0.8380952380952381,
 0.8412698412698413]

In [62]:
def dchr(l1,l2):
  s1 = 0
  s2 = 0
  for index, e in enumerate(l1):
    if index == 15 or index == 17:
      e = dict_transform(e)
      l2[index] = dict_transform(l2[index])
    s1 += e
    s2 += l2[index]
  n1 = -1
  n2 = -1
  if s1>=0 and s1<=9:
    n1 = 1
  elif s1>=10 and s1<=18:
    n1 = 2
  elif s1>=19 and s1<=29:
    n1 = 3
  else:
    n1 = 4
  
  if s2>=0 and s2<=9:
    n2 = 1
  elif s2>=10 and s2<=18:
    n2 = 2
  elif s2>=19 and s2<=29:
    n2 = 3
  else:
    n2 = 4
  
  if n1 == n2:
    return 1
  else:
    return 0

In [63]:
def adchr(all_alg_responses):
  adchr_list = []
  for all_responses  in all_alg_responses:
    s = 0
    for i in range(61,81):
      l1 = all_responses[str(i)]
      l2 = corr[str(i)]
      s += dchr(l1,l2)
    adchr_list.append(s/20)
  return adchr_list

In [64]:
adchr(all_alg_responses)

[0.3, 0.3, 0.3, 0.25, 0.15, 0.25, 0.25, 0.25]